# Sémantique

_Ismaël Bonneau_

#### Jouons un peu avec la sémantique...

Ce notebook a simplement pour but de jouer avec la sémantique des mots et de prendre en main et comparer plusieurs modèles de connaissance sur la sémantique.

- Le wordnet de spacy, qui utilise les données du wordnet de NLTK et qui s'installe avec la commande ***pip install spacy-wordnet***. https://spacy.io/

In [21]:
import os
import ast

import spacy
from spacy import displacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

nlp = spacy.load('en_core_web_sm')

dataset_path = "data"

#### On peut générer un arbre syntaxique avec les dépendances

In [24]:
with open(dataset_path + os.sep + "robust2004.txt", "r") as f:
    queries = ast.literal_eval(f.read())
queries = {d:queries[d][1] for d in queries}

In [43]:
displacy.render(nlp(queries['305']), style="dep", jupyter=True, options={'distance': 90})

In [11]:
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')